In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required libraries
!pip install node2vec
!pip install networkx



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1

In [3]:
import csv
import networkx as nx
from node2vec import Node2Vec
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

#take the embeddings generated using node2vec
#take anchor lookup table: maps g1 to g2
#prepare the mapping of the embeddings to another and
#make the ones without embeddings the test data
def prepare_the_Input_and_output_matrix(embeddings_1, embeddings_2, anchor_lookup_table):
    input_X = np.empty((0, embeddings_1[list(embeddings_1.keys())[0]].shape[0])) # Initialize empty numpy array for input data
    label_Y = np.empty((0, embeddings_2[list(embeddings_2.keys())[0]].shape[0])) # Initialize empty numpy array for labels
    test_X = np.empty((0, embeddings_1[list(embeddings_1.keys())[0]].shape[0])) # Initialize empty numpy array for test data
    corresponding_y_label_node_in_g2 = []
    unmarked_nodes_g1 = []
    marked_nodes_in_g1 = []
    #loop through the first node and its embeddings
    for node, embedding in embeddings_1.items():
        #check if node has a mapping
        #if it does, get its anchor node and its embedding
        if node in anchor_lookup_table:
            x_i = embedding.reshape(1, -1) # Reshape the embedding to be a row vector
            key_of_y_i = anchor_lookup_table[node]
            corresponding_y_label_node_in_g2.append(key_of_y_i)
            y_i = embeddings_2[key_of_y_i].reshape(1, -1) # Reshape the label embedding to be a row vector
            input_X = np.vstack((input_X, x_i)) # Append the row to input_X
            label_Y = np.vstack((label_Y, y_i)) # Append the row to label_Y
            marked_nodes_in_g1.append(node)
        #if not, this is the test embedding
        else:
            unmarked_nodes_g1.append(node)
            test_X = np.vstack((test_X, embedding.reshape(1, -1))) # Append the row to test_X
            #return input embedding, output embedding, test embedding and the actual node in the graph 2, unmarked nodes (not anchored)

    return input_X, label_Y, test_X, corresponding_y_label_node_in_g2, unmarked_nodes_g1, marked_nodes_in_g1

def prepare_data_for_knn_classifier(embeddings_2):
    X = []  # input features
    Y = []  # class labels

    for node, embedding in embeddings_2.items():
        x_i = np.array(embedding)  # Convert embedding to NumPy array
        y_i = int(node)  # Convert node label to integer

        X.append(x_i)
        Y.append(y_i)

    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y)

    return X, Y


def train_and_predict_knn_classifiers(X, Y, trained_embeddings_for_test_data, n_neighbors=5):
    # Initialize the KNN classifier
    knn_classifier = KNeighborsClassifier(n_neighbors=n_neighbors)

    # Train the KNN classifier
    knn_classifier.fit(X, Y)

    # Predict labels for the test data
    predicted_labels = knn_classifier.predict(trained_embeddings_for_test_data)

    return predicted_labels



def build_anchor_map(csv_file):
  anchor_map = {}
  with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      anchor_map[row[0]] = row[1]

  return anchor_map

def build_adjacency_list(csv_file):
    adjacency_list = {}
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            node = row[0]
            neighbor = row[1]
            if node not in adjacency_list:
                adjacency_list[node] = []
            adjacency_list[node].append(neighbor)
    return adjacency_list

def build_node2vec_embeddings(adjacency_list):
    # Create a graph from the adjacency list
    G = nx.Graph(adjacency_list)

    # Initialize Node2Vec
    node2vec = Node2Vec(G, dimensions=16, walk_length=12, num_walks=12, workers=10)

    # Generate node embeddings
    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    # Get the embeddings for all nodes
    embeddings = {node: model.wv[node] for node in G.nodes()}

    return embeddings


csv_file_g1 = '/content/drive/MyDrive/deep_learning_590/SP 2024 Deep Learning project description and files/g1_edgelist.csv'
csv_file_g2 = '/content/drive/MyDrive/deep_learning_590/SP 2024 Deep Learning project description and files/g2_edgelist.csv'
csv_anchor_file = '/content/drive/MyDrive/deep_learning_590/SP 2024 Deep Learning project description and files/map_anchors.csv'
adjacency_list_1 = build_adjacency_list(csv_file_g1)
adjacency_list_2 = build_adjacency_list(csv_file_g2)



In [4]:
embeddings_1 = build_node2vec_embeddings(adjacency_list_1)
embeddings_2 = build_node2vec_embeddings(adjacency_list_2)
anchor_table = build_anchor_map(csv_anchor_file)

X, Y, test, corresponding_y_label_node_in_g2, unmarked_nodes_g1, marked_nodes_in_g1 = prepare_the_Input_and_output_matrix(embeddings_1, embeddings_2, anchor_table)

Computing transition probabilities:   0%|          | 0/4941 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/4941 [00:00<?, ?it/s]

In [5]:
print(X.shape, Y.shape, test.shape)

(494, 16) (494, 16) (4447, 16)


In [6]:
print(len(embeddings_1), len(embeddings_2))

4941 4941


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture without non-linearity
class EmbeddingNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EmbeddingNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 100)
        self.fc3 = nn.Linear(100, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu( self.fc1(x))
        x = self.relu( self.fc2(x) )
        x = self.fc3(x)
        return x

# Define the MSE loss function
loss_function = nn.MSELoss()

# Convert numpy arrays to PyTorch tensors
input_tensors = torch.tensor(X, dtype=torch.float32)
target_tensors = torch.tensor(Y, dtype=torch.float32)
test_tensors = torch.tensor(test, dtype=torch.float32)

# Initialize the neural network model
input_dim = 16 # Dimensionality of input embeddings
hidden_dim = 128 # Dimensionality of hidden layer
output_dim = 16 # Dimensionality of output embeddings
model = EmbeddingNet(input_dim, hidden_dim, output_dim)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 80000
for epoch in range(num_epochs):
    # Forward pass
    output = model(input_tensors)

    # Compute the loss
    loss = loss_function(output, target_tensors)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Get the trained embeddings for the test data
embeddings_test = model(test_tensors).detach().numpy()

#get  embeddings for the train data
embeddings_train = model(input_tensors).detach().numpy()


# Print the trained embeddings for the test data
print("Trained embeddings for test data:")
print(embeddings_test)


Streaming output truncated to the last 5000 lines.
Epoch [30150/80000], Loss: 0.0006
Epoch [30160/80000], Loss: 0.0006
Epoch [30170/80000], Loss: 0.0008
Epoch [30180/80000], Loss: 0.0007
Epoch [30190/80000], Loss: 0.0006
Epoch [30200/80000], Loss: 0.0006
Epoch [30210/80000], Loss: 0.0006
Epoch [30220/80000], Loss: 0.0007
Epoch [30230/80000], Loss: 0.0008
Epoch [30240/80000], Loss: 0.0007
Epoch [30250/80000], Loss: 0.0007
Epoch [30260/80000], Loss: 0.0007
Epoch [30270/80000], Loss: 0.0008
Epoch [30280/80000], Loss: 0.0007
Epoch [30290/80000], Loss: 0.0007
Epoch [30300/80000], Loss: 0.0006
Epoch [30310/80000], Loss: 0.0006
Epoch [30320/80000], Loss: 0.0008
Epoch [30330/80000], Loss: 0.0006
Epoch [30340/80000], Loss: 0.0007
Epoch [30350/80000], Loss: 0.0007
Epoch [30360/80000], Loss: 0.0007
Epoch [30370/80000], Loss: 0.0007
Epoch [30380/80000], Loss: 0.0007
Epoch [30390/80000], Loss: 0.0006
Epoch [30400/80000], Loss: 0.0006
Epoch [30410/80000], Loss: 0.0007
Epoch [30420/80000], Loss: 0.00

In [16]:
#use knnn classifier to predict the class labels
X_KNN, Y_KNN = prepare_data_for_knn_classifier(embeddings_2)
#The nodes from graph 1 are predicted to map to the following nodes in graph 2
predicted_nodes_for_unmarked_nodes_in_g2_for_testing_data = train_and_predict_knn_classifiers(X_KNN, Y_KNN, embeddings_test, 1)

#train data predicting to measure overfitting and underfitting
predicted_nodes_for_unmarked_nodes_in_g2_for_train_data = train_and_predict_knn_classifiers(X_KNN, Y_KNN, embeddings_train, 1)

In [17]:
print(predicted_nodes_for_unmarked_nodes_in_g2_for_testing_data.shape, predicted_nodes_for_unmarked_nodes_in_g2_for_train_data.shape)
print("The umarked nodes have the following coreresponding nodes in graph 2")
print(predicted_nodes_for_unmarked_nodes_in_g2_for_testing_data)


(4447,) (494,)
The umarked nodes have the following coreresponding nodes in graph 2
[4512  643 1102 ... 1692  123  991]


In [18]:
print(len(marked_nodes_in_g1))
print(marked_nodes_in_g1)
print(len (list(predicted_nodes_for_unmarked_nodes_in_g2_for_train_data)))
print((list(predicted_nodes_for_unmarked_nodes_in_g2_for_train_data)))

494
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '32', '34', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '62', '63', '65', '66', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '88', '89', '90', '91', '92', '93', '95', '96', '97', '98', '99', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '115', '116', '117', '118', '119', '120', '121', '122', '124', '125', '126', '127', '128', '129', '132', '133', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '154', '155', '157', '158', '159', '160', '161', '162', '163', '164', '166', '168', '169', '170', '173', '174', '175', '178', '179', '180', '181', '183', '184', '185', '186', '187',

In [19]:
def sort_arrays(arr1, arr2):
    # Combine arr1 and arr2 into a list of tuples
    combined = list(zip(arr1, arr2))
    # Sort the combined list based on the values of arr1
    sorted_combined = sorted(combined, key=lambda x: x[0])
    # Unzip the sorted list to separate arr1 and arr2
    sorted_arr1, sorted_arr2 = zip(*sorted_combined)
    return sorted_arr1, sorted_arr2

# # Example usage:
# arr1 = [3, 1, 4, 2]
# arr2 = [30, 10, 40, 20]
# sorted_arr1, sorted_arr2 = sort_arrays(arr1, arr2)
# print(sorted_arr1)  # Output: (1, 2, 3, 4)
# print(sorted_arr2)  # Output: (10, 20, 30, 40)






In [20]:
import csv


def save_nodes_in_g1_and_predicted_anchor_nodes(unmarked_nodes_g1, unmarked_prediction_in_g2, marked_nodes_in_g1, marked_prediction_in_g2, path_file):
    # Combine unmarked and marked nodes
    all_nodes_g1 = unmarked_nodes_g1 + marked_nodes_in_g1
    all_prediction_in_g2 = list(unmarked_prediction_in_g2) + list(marked_prediction_in_g2)

    # all_nodes_g1, all_prediction_in_g2 = sort_arrays(all_nodes_g1_p, all_prediction_in_g2_p)

    # Prepare data for writing to CSV
    data = zip(all_nodes_g1, all_prediction_in_g2)

    # Write data to CSV file
    with open(path_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write headers
        writer.writerow(['g1', 'g2'])
        # Write data
        writer.writerows(data)


# Example usage
path_file = '/content/drive/MyDrive/deep_learning_590/SP 2024 Deep Learning project description and files/nodes_predictions.csv'
save_nodes_in_g1_and_predicted_anchor_nodes(unmarked_nodes_g1, predicted_nodes_for_unmarked_nodes_in_g2_for_testing_data,marked_nodes_in_g1,  predicted_nodes_for_unmarked_nodes_in_g2_for_train_data,path_file)


In [21]:
from sklearn.metrics import accuracy_score
print(len(marked_nodes_in_g1), len(predicted_nodes_for_unmarked_nodes_in_g2_for_train_data))

# print(Y.shape)
# Calculate accuracy
accuracy = accuracy_score(marked_nodes_in_g1, predicted_nodes_for_unmarked_nodes_in_g2_for_train_data)

print("Accuracy on training data:", accuracy)



494 494
Accuracy on training data: 0.0
